# Imports and NLP


In [1]:
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  !pip install fuzzywuzzy[speedup]
else:
  print("Not running in colab")

Running on CoLab
     |████████████████████████████████| 51kB 8.8MB/s 
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149816 sha256=f4a73151d2ce5f77b0a470c833b03de93358377dbec5cc0e09d0556937a333f8
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-levenshtein


In [2]:
!pip install colab_everything

     |████████████████████████████████| 747kB 22.5MB/s 
     |████████████████████████████████| 133kB 53.5MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.3-cp37-none-any.whl size=18979 sha256=31c4575a1ad6cf8df140687e7714c8db5f5cfa454607282ea9f48c89acefe30b
  Stored in directory: /root/.cache/pip/wheels/f0/77/01/d23761888c52099d1817121dc1f28afbc1c7cbf3caac93b1ff
Successfully built pyngrok
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
import ast
import itertools
import numpy as np
import pandas as pd 
import pickle
import re
import spacy
import spacy.cli
import time
from colab_everything import ColabCustom
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
from time import process_time, gmtime, strftime

In [4]:
%load_ext google.colab.data_table

In [5]:
spacy.cli.download("en_core_web_lg")
nlp = spacy.load("en_core_web_lg")

⚠ Skipping model package dependencies and setting `--no-deps`. You
don't seem to have the spaCy package itself installed (maybe because you've
built from source?), so installing the model dependencies would cause spaCy to
be downloaded, which probably isn't what you want. If the model package has
other dependencies, you'll have to install them manually.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


# Examine Data

In [6]:
url = "https://raw.githubusercontent.com/vjmiyagi/DS_BuildWeek_Dec_2020/main/csv/data.csv"
df_main = pd.read_csv(url)
df_artist = pd.read_csv("https://raw.githubusercontent.com/vjmiyagi/DS_BuildWeek_Dec_2020/main/csv/data_by_artist.csv")
df_genre = pd.read_csv("https://raw.githubusercontent.com/vjmiyagi/DS_BuildWeek_Dec_2020/main/csv/data_by_genres.csv")
df_year = pd.read_csv("https://raw.githubusercontent.com/vjmiyagi/DS_BuildWeek_Dec_2020/main/csv/data_by_year.csv")
df_w = pd.read_csv("https://raw.githubusercontent.com/vjmiyagi/DS_BuildWeek_Dec_2020/main/csv/data_w_genres.csv")

In [7]:
df_main.shape

(170653, 19)

In [8]:
a = df_main.head()
b = df_main.tail()
c = df_artist.head()
d = df_artist.tail()
e = df_genre.head()
f = df_genre.tail()
g = df_year.head()
h = df_year.tail()
i = df_year.head()
j = df_year.tail()

heads = [a, c, e, f, i]
tails = [b, d, f, h, j]
print(heads)
print(tails)

[   valence  year  acousticness  ... release_date  speechiness    tempo
0   0.0594  1921         0.982  ...         1921       0.0366   80.954
1   0.9630  1921         0.732  ...         1921       0.4150   60.936
2   0.0394  1921         0.961  ...         1921       0.0339  110.339
3   0.1650  1921         0.967  ...         1921       0.0354  100.109
4   0.2530  1921         0.957  ...         1921       0.0380  101.665

[5 rows x 19 columns],    mode  count  acousticness  ...   valence  popularity  key
0     1      9      0.590111  ...  0.389500   38.333333    5
1     1     26      0.862538  ...  0.268865   30.576923    5
2     1      7      0.856571  ...  0.354857   34.857143    0
3     1     27      0.884926  ...  0.372030   34.851852    0
4     1      7      0.510714  ...  0.482286   43.000000    5

[5 rows x 15 columns],    mode                  genres  acousticness  ...   valence  popularity  key
0     1  21st century classical      0.979333  ...  0.103783   27.833333    6
1  

In [9]:
df = df_main
df

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.98200,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.6650,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.73200,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.1600,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.96100,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.1010,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.96700,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.3810,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.95700,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.2290,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170648,0.6080,2020,0.08460,"['Anuel AA', 'Daddy Yankee', 'KAROL G', 'Ozuna...",0.786,301714,0.808,0,0KkIkfsLEJbrcIhYsCL7L5,0.000289,7,0.0822,-3.702,1,China,72,2020-05-29,0.0881,105.029
170649,0.7340,2020,0.20600,['Ashnikko'],0.717,150654,0.753,0,0OStKKAuXlxA0fMH54Qs6E,0.000000,7,0.1010,-6.020,1,Halloweenie III: Seven Days,68,2020-10-23,0.0605,137.936
170650,0.6370,2020,0.10100,['MAMAMOO'],0.634,211280,0.858,0,4BZXVFYCb76Q0Klojq4piV,0.000009,4,0.2580,-2.226,0,AYA,76,2020-11-03,0.0809,91.688
170651,0.1950,2020,0.00998,['Eminem'],0.671,337147,0.623,1,5SiZJoLXp3WOl3J4C8IK0d,0.000008,2,0.6430,-7.161,1,Darkness,70,2020-01-17,0.3080,75.055


In [10]:
df["artists"]

0         ['Sergei Rachmaninoff', 'James Levine', 'Berli...
1                                            ['Dennis Day']
2         ['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...
3                                          ['Frank Parker']
4                                            ['Phil Regan']
                                ...                        
170648    ['Anuel AA', 'Daddy Yankee', 'KAROL G', 'Ozuna...
170649                                         ['Ashnikko']
170650                                          ['MAMAMOO']
170651                                           ['Eminem']
170652                                ['KEVVO', 'J Balvin']
Name: artists, Length: 170653, dtype: object

In [11]:
print("df:", df.shape)
print(df.head(1))
print("df_artist:", df_artist.shape)
print(df_artist.head(1))
print("df_genre:", df_genre.shape)
print(df_genre.head(1))
print("df_year:", df_year.shape)
print(df_year.head(1))
print("df_w:", df_w.shape)
print(df_w.head(1))

df_w.tail(1)

df: (170653, 19)
   valence  year  acousticness  ... release_date  speechiness   tempo
0   0.0594  1921         0.982  ...         1921       0.0366  80.954

[1 rows x 19 columns]
df_artist: (28680, 15)
   mode  count  acousticness  ... valence  popularity  key
0     1      9      0.590111  ...  0.3895   38.333333    5

[1 rows x 15 columns]
df_genre: (2973, 14)
   mode                  genres  acousticness  ...   valence  popularity  key
0     1  21st century classical      0.979333  ...  0.103783   27.833333    6

[1 rows x 14 columns]
df_year: (100, 14)
   mode  year  acousticness  ...   valence  popularity  key
0     1  1921      0.886896  ...  0.379327    0.653333    2

[1 rows x 14 columns]
df_w: (28680, 16)
           genres                           artists  ...  mode  count
0  ['show tunes']  "Cats" 1981 Original London Cast  ...     1      9

[1 rows x 16 columns]


,genres,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count
28679,['classic korean pop'],조정현,0.568,0.447,237688.0,0.215,0.000001,0.0649,-16.478,0.0272,71.979,0.177,31.0,10,1,2


# Get to work with the main DataFrame, df, and focus on text fields

In [12]:
# Convert string to list
df.artists = df.artists.apply(ast.literal_eval)
# returning a list store in a string, str('[a,g,c]') --> list [a,g,c]
print("Artist Column Type: ", type(df.artists[0]))
df['artists']

Artist Column Type:  <class 'list'>


0         [Sergei Rachmaninoff, James Levine, Berliner P...
1                                              [Dennis Day]
2         [KHP Kridhamardawa Karaton Ngayogyakarta Hadin...
3                                            [Frank Parker]
4                                              [Phil Regan]
                                ...                        
170648    [Anuel AA, Daddy Yankee, KAROL G, Ozuna, J Bal...
170649                                           [Ashnikko]
170650                                            [MAMAMOO]
170651                                             [Eminem]
170652                                    [KEVVO, J Balvin]
Name: artists, Length: 170653, dtype: object

In [13]:
def lst_join(list_artists):
  """transform from list to string
  to get rid of the [] of the list"""
  Jointed_list = ", ".join(list_artists)
  return Jointed_list

df.artists = df.artists.apply(lst_join)
df[['artists','id']].head()

,artists,id
0,"Sergei Rachmaninoff, James Levine, Berliner Ph...",4BJqT0PrAfrxzMOxytFOIz
1,Dennis Day,7xPhfUan2yNtyFG0cUWkt8
2,KHP Kridhamardawa Karaton Ngayogyakarta Hadini...,1o6I8BglA6ylDMrIELygv1
3,Frank Parker,3ftBPsC5vPBKxYSee08FDH
4,Phil Regan,4d6HGyGT8e121BsdKmw9v6


In [14]:
df['name_artist'] = df['name'] + ", " + df['artists']
df[['name_artist']].head()

,name_artist
0,"Piano Concerto No. 3 in D Minor, Op. 30: III. ..."
1,"Clancy Lowered the Boom, Dennis Day"
2,"Gati Bali, KHP Kridhamardawa Karaton Ngayogyak..."
3,"Danny Boy, Frank Parker"
4,"When Irish Eyes Are Smiling, Phil Regan"


In [15]:
#then = time.time() #Time before the operations start
#print(strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime()))
#x = [nlp(str(item)).vector for item in df['name']]
#now = time.time() #Time after it finished
#z = (now-then)/60
#print(f"It took: {z:0.1f} minutes vectorize name column")
#df_name = pd.DataFrame(x)
#print(df2.shape)
#df_name.head()

In [16]:
then = time.time() #Time before the operations start
print(strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime()))
x = [nlp(str(item)).vector for item in df['name_artist']]
now = time.time() #Time after it finished
z = (now-then)/60
print(f"It took: {z:0.1f} minutes to vectorize name_artist column")
df_song_artist = pd.DataFrame(x)
print(df_song_artist.shape)
df_song_artist.head()

Fri, 05 Mar 2021 03:15:49 +0000
It took: 17.8 minutes to vectorize name_artist column
(170653, 300)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,-0.104343,0.256664,-0.000575,-0.123706,0.105237,0.071267,0.105415,-0.218627,0.005914,0.844697,-0.221438,0.156455,0.026264,0.030606,-0.140692,0.068536,0.117969,0.855142,-0.109056,-0.001974,0.072097,0.122346,0.017775,-0.085537,0.057242,0.026935,-0.226265,0.050306,0.309934,0.060790,0.061881,-0.081949,-0.090020,0.099509,0.033985,-0.039915,0.041412,-0.007238,-0.001329,-0.035478,...,0.124067,0.065372,-0.008878,0.168204,-0.066596,-0.140428,0.105005,-0.096926,-0.031097,0.056340,-0.217942,-0.113236,-0.022624,0.002774,-0.105052,0.097090,0.131393,-0.130299,-0.072240,0.139426,0.266036,-0.163241,0.159167,-0.192002,-0.102537,-0.001253,-0.050798,-0.211696,0.099663,-0.115656,0.211048,-0.040847,-0.069745,-0.008876,0.224507,0.073500,-0.034220,-0.063754,-0.210584,0.030777
1,-0.110043,0.270508,-0.017440,-0.060651,0.316853,0.039507,-0.041189,-0.027936,0.023999,1.406347,-0.076667,-0.308565,0.289800,0.039321,-0.074606,-0.094344,-0.081386,0.721140,-0.153476,0.055845,0.069611,0.001416,0.141988,-0.147071,-0.001511,-0.116420,-0.266695,0.137878,-0.025549,0.070947,0.010973,-0.020686,0.046662,0.207328,0.106267,-0.101173,-0.115103,0.051799,0.054557,-0.006665,...,0.070192,0.008541,-0.135586,0.105572,0.007952,-0.314307,0.097951,-0.072360,-0.182626,0.002562,0.080890,-0.098343,0.094457,0.064836,0.000665,-0.155242,-0.005561,0.191624,0.077312,0.033754,-0.007187,-0.151586,-0.137674,-0.091476,0.086862,0.007816,0.043054,-0.213127,0.109522,0.044134,-0.187479,0.182865,0.123634,-0.043874,0.015906,-0.132141,-0.099888,-0.006539,-0.181266,-0.124857
2,0.068590,0.113422,0.103174,0.038538,0.106071,0.088705,0.072574,-0.155586,0.002221,0.001743,-0.118041,0.039472,0.041700,0.005835,-0.017671,0.018833,0.012381,0.150369,0.148339,0.073717,-0.019314,0.195209,0.018395,0.000544,-0.048497,-0.052766,-0.024003,-0.043881,-0.004486,0.166690,0.113390,-0.169941,-0.053533,0.020867,-0.126260,-0.112777,0.016822,0.002729,-0.153207,-0.014552,...,0.131772,-0.095836,0.076595,0.070588,-0.014139,-0.024047,0.035469,0.036178,-0.153459,0.019641,-0.275619,-0.116924,0.066542,0.062275,-0.050696,-0.000262,0.057690,0.014037,0.169320,0.062680,0.052975,-0.162852,0.066968,-0.093190,-0.019099,-0.045745,-0.047715,-0.165826,0.078896,-0.022478,0.051376,-0.017264,0.000504,0.085651,0.046241,0.034768,-0.003681,-0.013761,-0.016225,0.021574
3,-0.292194,0.387879,-0.102738,0.051408,0.057164,0.021752,0.242027,-0.276544,-0.029607,1.168303,-0.139846,-0.250231,-0.026075,-0.171859,-0.346144,0.139051,0.030024,0.482731,0.129428,0.087337,0.008702,-0.105756,0.003223,-0.240241,-0.020710,-0.064412,-0.271561,0.211825,0.224007,-0.024155,0.052121,0.193486,-0.145867,0.257438,-0.164728,-0.129346,-0.072548,0.062923,-0.073355,-0.010203,...,-0.079429,-0.026412,-0.232371,0.142913,-0.112252,-0.249662,-0.003790,0.195182,-0.475334,0.271938,-0.593094,0.148620,0.103629,0.318412,-0.116806,-0.110854,0.159057,-0.258703,-0.075874,0.031026,0.104132,-0.017178,-0.013615,0.214068,0.253996,0.014365,0.123764,-0.001096,-0.074394,-0.286461,0.212364,-0.084463,-0.022306,0.056945,0.207422,-0.061910,-0.045042,-0.193597,-0.233584,0.041322
4,-0.032911,0.133576,-0.197918,-0.142909,0.181914,-0.181740,0.051254,-0.082904,-0.087125,1.545840,-0.121138,0.068509,0.018185,-0.203634,-0.098416,-0.128725,-0.050222,0.503940,-0.142028,-0.087403,0.032555,-0.013032,-0.048948,-0.219706,-0.031451,-0.160643,-0.315007,0.002218,0.024770,-0.071185,-0.094831,-0.147687,0.010822,0.118725,0.029225,-0.071990,-0.124607,-0.045435,-0.186694,-0.057256,...,-0.074111,0.123906,-0.127728,0.188096,0.080501,-0.009474,0.008095,0.125152,-0.141503,0.232485,-0.425659,0.010340,-0.034684,0.229545,-0.018984,0.049981,0.069445,-0.022785,-0.058277,-0.111073,0.209595,0.178669,-0.202245,0.124928,0.203760,-0.032610,0.093533,-0.029013,0.1053

In [17]:
our_vectors = df_song_artist

In [18]:
print("df_song_artist Sum of null values")
print(df_song_artist.isnull().sum())

df_song_artist Sum of null values
0      0
1      0
2      0
3      0
4      0
      ..
295    0
296    0
297    0
298    0
299    0
Length: 300, dtype: int64


# Fitting the Model !

In [19]:
algorithm = "ball_tree"

leaf_size = 15

metric = "minkowski"

metric_params = "None"

n_jobs =  -1

n_neighbors = 10

nn = NearestNeighbors(algorithm=algorithm, leaf_size=leaf_size, 
                      n_neighbors=n_neighbors, n_jobs=n_jobs)


model = nn.fit(df_song_artist)

In [25]:
#Testing the model results

test = "love songs"
#model = model


test_vect = nlp(test).vector
vect = test_vect.reshape(1, -1)
results = nn.kneighbors(vect)[1].tolist()[0]

s = list([df['name'].iloc[x] for x in results])
a = [df['artists'].iloc[x] for x in results]
y = [df['release_date'].iloc[x] for x in results]
   
cn = ["Song", "Artist", "Year", "Row #"]
print(f"{cn[0]:<51}{cn[1]:<45}{cn[2]:<15}{cn[3]:^25}")
print("=" * 156)
for i in range(n_neighbors-1):
    print(f"{s[i][0:50]:<51}{a[i][0:44]:<45}{y[i]:^15}{results[i]:>16}")

Song                                               Artist                                       Year                     Row #          
Love Song                                          Madonna                                        1989-03-21             164461
Hello Song                                         Music Together                                 2007-10-29             122226
Silly Love Songs                                   Wings                                          2001-05-07              71267
Silly Love Songs                                   Wings                                          1987-11-02             163991
Silly Love Songs                                   Wings                                          1978-11-22             100303
Love Song                                          Simple Minds                                   1981-09-12              84584
Love Song                                          311                                         

In [21]:
filename = 'model'
pickle.dump(model, open(filename, 'wb'))

In [22]:
import zipfile
with zipfile.ZipFile("temp.zip", "w", zipfile.ZIP_DEFLATED) as zip_file:
     zip_file.write("model")